---
## Visualize extracted poles

In order to verify the results and spot potential issues, we can visualize individual objects. The visualization renders the object from two directions (X and Y axis), along with a 3D projection.

In [ ]:
import set_path
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely.geometry as sg
import laspy
from upc_analysis import visualization
import multiprocessing

In [ ]:
# Paths - point cloud data
base_folder = '/home/azureuser/cloudfiles/code/blobfuse/ovl'
dataset_folder = f'{base_folder}/pointcloud/Unlabeled/Amsterdam/nl-amsd-200923-7415-laz/las_processor_bundled_out/'  # folder with point clouds
pred_folder = f'{base_folder}/predictions/nl-amsd-200923-7415-laz/'  # folder with predictions as npz files
prefix = 'filtered_'
prefix_pred = 'pred_'

# Paths - extracted poles
base_folder_output = '/home/azureuser/cloudfiles/code/blobfuse/ovl'
output_file_filter = base_folder + "/20230707-150648_poles_extracted_zuidoost_filtered.csv"
img_out_folder = f'{base_folder}/images/20230707-150648/'

#### Load poles data

In [ ]:
poles_df = pd.read_csv(output_file_filter)
poles_df.sort_values('tilecode', inplace=True)
poles_df.shape

#### Loop over all poles

In [ ]:
# Save png of object x, y and 3d axis
open_tile = []
for _, obj in poles_df.iterrows():
    # Get object location and top (per pole)
    identifier = obj.identifier
    obj_location = (obj.rd_x, obj.rd_y, obj.z)
    obj_top = (obj.tx, obj.ty, obj.tz)

    if obj.tilecode != open_tile:
        # Get the point cloud data (per tile)
        cloud = laspy.read(f'{dataset_folder}{prefix}{obj.tilecode}.laz')
        points = np.vstack((cloud.x, cloud.y, cloud.z)).T
        npz_file = np.load(pred_folder + prefix_pred + obj.tilecode + '.npz')
        labels = npz_file['label']
        colors = np.vstack((cloud.red, cloud.green, cloud.blue)).T / (2**16 - 1)
        open_tile = obj.tilecode  # tile_code that is currently open

    # Get a mask for the point cloud around the object's location (per pole)
    obj_mask = visualization.get_mask_for_obj(points, obj_location, obj_top[2])
    if sum(obj_mask) > 0:
        # Save the object for all axes
        write_path = img_out_folder + 'object_all_axes' 
        p = multiprocessing.Process(target=visualization.generate_png_all_axes, 
        args=(identifier, points[obj_mask], labels[obj_mask], write_path, 
        colors[obj_mask], np.vstack((obj_location, obj_top)), False))
        p.start()                  
        # Save the objects per axis
        write_path = img_out_folder + 'object_per_axis'
        p = multiprocessing.Process(target=visualization.generate_png_single_axis, 
        args=(identifier, points[obj_mask], labels[obj_mask], write_path, colors[obj_mask], 'x'))
        p.start() 
        p = multiprocessing.Process(target=visualization.generate_png_single_axis, 
        args=(identifier, points[obj_mask], labels[obj_mask], write_path, colors[obj_mask], 'y'))
        p.start() 

#### Look at one pole

In [ ]:
# Either choose a row number of sample one randomly
idx = 7
# idx = np.random.randint(0, len(poles_df))
# row_id = poles_df.index[idx]

# Get the object features
obj = poles_df.loc[idx]
obj_location = (obj.rd_x, obj.rd_y, obj.z)
obj_top = (obj.tx, obj.ty, obj.tz)

# Let's see the object in question
poles_df.loc[[idx]]

In [ ]:
# Print location (to be pasted in Google Maps)
my_lon = float(poles_df.loc[[idx]]['rd_x'])
my_lat = float(poles_df.loc[[idx]]['rd_y'])
df_loc = gpd.GeoDataFrame({'geometry': [sg.Point(my_lon, my_lat)]}, crs="epsg:28992").to_crs("epsg:4326") 
print(float(df_loc['geometry'].y), float(df_loc['geometry'].x))

In [ ]:
# Save png of object x, y and 3d axis
obj_location = (obj.rd_x, obj.rd_y, obj.z)
obj_top = (obj.tx, obj.ty, obj.tz)
cloud = laspy.read(f'{dataset_folder}{prefix}{obj.tilecode}.laz')
points = np.vstack((cloud.x, cloud.y, cloud.z)).T
npz_file = np.load(pred_folder + prefix_pred + obj.tilecode + '.npz')
labels = npz_file['label']
colors = np.vstack((cloud.red, cloud.green, cloud.blue)).T / (2**16 - 1)

# Get a mask for the point cloud around the object's location
obj_mask = visualization.get_mask_for_obj(points, obj_location, obj_top[2])
if sum(obj_mask) > 0:
    # Show and save the object
    write_path = img_out_folder + 'object_all_axes' 
    visualization.generate_png_all_axes(idx, points[obj_mask], labels[obj_mask], 
                        write_path, colors=colors[obj_mask], estimate=np.vstack((obj_location, obj_top)),
                        show_image=True)